In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import matplotlib.pyplot as plt
import math
import cv2

In [ ]:
import torch
import random
import torchvision
from torchvision.datasets import MNIST  ## load MNIST dataset from torchvision
import torchvision.transforms as transforms

In [ ]:
trainset = MNIST(root='data/', download=True, transform=transforms.Compose([transforms.ToTensor()]))
testset = MNIST(root='data/', train=False, transform=transforms.Compose([transforms.ToTensor()]))

print("0th image data shape: ", trainset[0][0].shape) ## 0th image data (채널,가로,세로)
print("0th label : ", trainset[0][1]) ## label
image, label = trainset[0]



In [ ]:
# visualize
for i in range(9):
   random_index = random.randint(1,10000)
   image, label = trainset[random_index]
   ax = plt.subplot(3, 3, i + 1)
   plt.imshow(image.permute(1,2,0).numpy())
   plt.title(label)
   plt.axis("off")


In [ ]:
# 교수님 버전 특징 추출--일부만
b_image = torch.where(image>0.5, 1, 0)


In [ ]:
# 면적: 해당 영역에 속하는 픽셀의 개수-->20개의 픽셀로 구성
# 중심: 영역의 중심 좌표. 해당 픽셀의 평균 위치
# 행 분산: x방향으로 퍼져 있는 정도. 값이 클 수록 가로로 더 퍼져 있음
# 열 분산: y방향으로 퍼져 있는 정도. 값이 클 수록 세로로 더 퍼져 있음
# 혼합 분산: 행과 열의 공분산. 양수면 오른쪽 위 또는 왼쪽 아래로 퍼져있음. 음수면 왼쪽 위 또는 오른쪽 아래로 퍼져있음. 0에 가까우면 방향성과 상관없이 균형잡힘
# 둘레 : 영역들을 둘러싸는 픽셀들의 총길이
# 둥근 정도: 영역이 얼마나 원형에 가까운지. 1에 가까울수록 원형, 작을 수록 길쭉하거나 복잡한 형태.

def make_binary_img(img, T):
  binary_image = img.copy()

  binary_image[img>=T]=1
  binary_image[img<T]=0

  return binary_image
# 특징 추출 함수
def extract_features(image_tensor):
    # 이진화 진행 시 픽셀이 1인 영역만 추적하면 되므로 연산이 단순해짐
    image = image_tensor.squeeze().numpy() #[1,28,28]->[28,28]변환
    binary_img = make_binary_img(image, 0.5)
    #image_bin = (image > 0.5).astype(np.uint8)  # 이진화

    coords = np.column_stack(np.where(image_bin > 0)) # 픽셀값이 1인 좌표들 위치
    if coords.shape[0] == 0:
        return np.zeros(8)  # 빈 이미지(숫자 없는 경우) 처리

    a = coords.shape[0] # a = 픽셀 수

    # 행분산, 열분산, 혼합 분산
    y_bar, x_bar = np.mean(coords, axis=0)
    vr = np.mean((coords[:, 1] - x_bar) ** 2)  # x 방향 분산
    vc = np.mean((coords[:, 0] - y_bar) ** 2)  # y 방향 분산
    vrc = np.mean((coords[:, 0] - y_bar) * (coords[:, 1] - x_bar))  # 혼합 분산

    # 둘레
    contours, _ = cv2.findContours(image_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    p = sum(cv2.arcLength(cnt, True) for cnt in contours)

    r = (4 * np.pi * a) / (p ** 2) if p != 0 else 0

    return np.array([a, y_bar, x_bar, vr, vc, vrc, p, r])

In [ ]:
features = []
labels = []
sample_size = 1000
for i in range(sample_size):
    image, label = trainset[i]
    f = extract_features(image)
    features.append(f)
    labels.append(label)

features = np.array(features)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
# K-means 클러스터링
k = 10
kmeans = KMeans(n_clusters=k, random_state=0, n_init=10)
clusters = kmeans.fit_predict(features)

# 군집 균일도 측정 (Silhouette Score)
score = silhouette_score(features, clusters)
print(f"Silhouette Score: {score:.4f}")

In [ ]:
# 시각화 (2D 축소: PCA or t-SNE 추천 가능)
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
reduced = pca.fit_transform(features)

plt.figure(figsize=(8, 6))
for i in range(k):
    plt.scatter(reduced[clusters == i, 0], reduced[clusters == i, 1], label=f'Cluster {i}', alpha=0.6)
plt.title("K-means Clustering of MNIST Feature Vectors")
plt.legend()
plt.show()

In [ ]:
# uniformity 측정-클러스터의 분산이 작을수록 더 응집되어 있음
def measure_cluster_uniformity(features, clusters, kmeans, k=10):
    uniformities = []

    for i in range(k):
        cluster_points = features[clusters == i]
        centroid = kmeans.cluster_centers_[i]
        # 유클리드 제곱 거리의 평균
        intra_distances = np.linalg.norm(cluster_points - centroid, axis=1) ** 2
        variance = np.mean(intra_distances)
        uniformities.append(variance)
        print(f"Cluster {i}: Variance (lower is better) = {variance:.4f}")

    return uniformities

uniformities = measure_cluster_uniformity(features, clusters, kmeans, k=10)


In [ ]:
# 각 클러스터별 실루엣 계수 측정. 범위: -1~1 높을수록 더 균일
from sklearn.metrics import silhouette_samples

sil_samples = silhouette_samples(features, clusters)

for i in range(k):
    cluster_silhouettes = sil_samples[clusters == i]
    avg_silhouette = np.mean(cluster_silhouettes)
    print(f"Cluster {i}: Silhouette score (higher is better) = {avg_silhouette:.4f}")


In [ ]:
# 균일도 시각화
plt.boxplot([sil_samples[clusters == i] for i in range(k)], labels=[f"C{i}" for i in range(k)])
plt.title("Silhouette Score per Cluster")
plt.ylabel("Silhouette Score")
plt.show()